<a href="https://colab.research.google.com/github/NicolasFranklin-MM/GHPT-colab-experiments-NickMM/blob/main/Grasshopper_testing-Nick2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Install
install openai and setup API key
API key must be saved in Colab sectrets as OPENAI_API_KEY

In [24]:
!pip install openai

In [25]:
!pip install -U langsmith

In [26]:
import os
from google.colab import userdata
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = f"GHPT_Baseline"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = userdata.get("LANGCHAIN_API_KEY")

In [27]:
from openai import OpenAI
from langsmith.wrappers import wrap_openai
from langsmith import traceable

In [28]:
OPENAI_API_KEY=userdata.get("OPENAI_API_KEY")
# OPENAI_API_KEY

# OpenAI

In [29]:
#client = OpenAI(api_key=OPENAI_API_KEY)
client = wrap_openai(OpenAI(api_key=OPENAI_API_KEY))
model="gpt-3.5-turbo-1106"
temperature=0

In [30]:
# create the openai api call function
def call_openai(prompt: str, system_prompt: str = "", model: str = "gpt-3.5-turbo-1106", temperature: float = 0):
    completion = client.chat.completions.create(
        model=model,
        temperature=temperature,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
    )

    return completion

In [31]:
model: str = "gpt-3.5-turbo-1106"
# model: str = "gpt-3.5-turbo-instruct"
temperature: float = 0

prompt="say this is a test"
system_prompt=""

#completion = call_openai(prompt=prompt, system_prompt=system_prompt, model=model, temperature=temperature)
#result = completion.choices[0].message.content
#print(result)

# GHPT

## prompt template

In [32]:
system_prompt = """
You are a Grasshopper Expert and are going to help create Grasshopper Definitions.
Keep the answers short and concise.
Make sure you create and connect a component for every non-optional input
If you're not sure about the answer, but think there's additional information that could help you, please ask for that information.

Always use the given format, avoid any devitation.
"""

In [33]:
prompt_template = r"""
// Question : How do I add two numbers inside of Grasshopper?
// Reasoning: To add two numbers we need to Add. There is an Addition component that performs this function. We need to create two numbers the user can edit, we can use the Number Slider for both numbers. And then we can Connections all of the components together
// JSON:
{
	"Advice": "Make sure to set the number sliders to the correct value",
	"Additions": [
		{
			"Name": "Number Slider",
			"Id": 1, // This number must be unique
			"value": "0..25..100" // This is optional
		},
		{
			"Name": "Number Slider",
			"Id": 2,
			"value": "-50..25..100" // This is optional
		},
		{
			"Name": "Addition",
			"Id": 3
		}
	],
	"Connections": [
		{
			"To": { // Addition
				"Id": 3,
				"ParameterName": "A"
			},
			"From": { // Number Slider,
				"Id": 1,
				"ParameterName": "number"
			}
		},
		{
			"To": { // Addition
				"Id": 3,
				"ParameterName": "B"
			},
			"From": { // Number Slider,
				"Id": 2,
				"ParameterName": "number"
			}
		}
	]
}

// Question: How do I create a cup shape in grasshopper
// Reasoning: To create a cup shape in Grasshopper, we"ll first create a circle using a "Circle CNR" component, which will act as the base of the cup. Then, we will use a "Move" component to move the base circle vertically to create the upper rim of the cup. To create the body of the cup, we"ll loft these two circles using a "Loft" component.
// JSON:
{
	"Advice": "Remember, to properly define the cup\"s dimensions using the number slider",
	"Additions": [
		{
			"Name": "Circle CNR",
			"Id": 1
		},
		{
			"Name": "Move",
			"Id": 2
		},
		{
			"Name": "Loft",
			"Id": 3
		}
	],
	"Connections": [
		{
			"To": {
				"Id": 2,
				"ParameterName": "Geometry"
			},
			"From": {
				"Id": 1,
				"ParameterName": "Circle"
			}
		},
		{
			"To": {
				"Id": 3,
				"ParameterName": "Curves"
			},
			"From": {
				"Id": 1,
				"ParameterName": "Circle"
			}
		},
		{
			"To": {
				"Id": 3,
				"ParameterName": "Curves"
			},
			"From": {
				"Id": 2,
				"ParameterName": "Geometry"
			}
		}
	]
}
```

// Question: How do I create a twisty skyscraper?
// Reasoning: We can create the "twist" using the twist component, so lets extrude a rectangle and twist it!
// JSON:
{
	"Advice": "Make sure to use reasonable inputs or the skyscraper will look weird",
	"Additions": [
		{
			"Name": "Rectangle",
			"Id": 1
		},
		{
			"Name": "Number Slider",
			"Id": 2,
			"Value": "0..50..100"
		},
		{
			"Name": "Unit Z",
			"Id": 3
		},
		{
			"Name": "Extrusion",
			"Id": 4
		},
		{
			"Name": "Number Slider",
			"Id": 5,
			"Value": "0..90..360"
		},
		{
			"Name": "Line",
			"Id": 6
		},
		{
			"Name": "Point"
			"Id": 7,
			"value": "{0,0,0}"
		},
		{
			"Name": "Point"
			"Id": 8,
			"value": "{0,0,250}"
		},
		{
			"Name": "Twist",
			"Id": 9
		},
		{
			"Name": "Solid Union",
			"Id": 10
		},
		{
			"Name": "Brep Join",
			"Id": 11
		}
	],
	"Connections": [
		{
			"To": {
				"Id": 4,
				"ParameterName": "Base"
			},
			"From": {
				"Id": 1,
				"ParameterName": "Rectangle"
			}
		},
		{
			"To": {
				"Id": 3,
				"ParameterName": "Factor"
			},
			"From": {
				"Id": 2,
				"ParameterName": "Number"
			}
		},
		{
			"To": {
				"Id": 9,
				"ParameterName": "Angle"
			},
			"From": {
				"Id": 5,
				"ParameterName": "Number"
			}
		},
		{
			"To": {
				"Id": 4,
				"ParameterName": "Direction"
			},
			"From": {
				"Id": 3,
				"ParameterName": "Unit vector"
			}
		},
		{
			"To": {
				"Id": 9,
				"ParameterName": "Geometry"
			},
			"From": {
				"Id": 4,
				"ParameterName": "Extrusion"
			}
		},
		{
			"To": {
				"Id": 6,
				"ParameterName": "Start Point"
			},
			"From": {
				"Id": 7,
				"ParameterName": "Point"
			}
		},
		{
			"To": {
				"Id": 6,
				"ParameterName": "End Point"
			},
			"From": {
				"Id": 8,
				"ParameterName": "Point"
			}
		},
		{
			"To": {
				"Id": 10,
				"ParameterName": "Breps"
			},
			"From": {
				"Id": 9,
				"ParameterName": "Geometry"
			}
		},
		{
			"To": {
				"Id": 11,
				"ParameterName": "Breps"
			},
			"From": {
				"Id": 10,
				"ParameterName": "Result"
			}
		},
		{
      "To": {
          "Id": 9,
          "ParameterName": "Axis"
        },
      "From": {
          "Id": 6,
          "ParameterName": "Line"
      }
		}
	]
}

// Question: {QUESTION}
// Reasoning:
"""

In [34]:
prompt_template = prompt_template.replace("{","{{").replace("}","}}").replace("{{QUESTION}}","{QUESTION}")

In [35]:
prompt_template_error = """The above json returned the error: {ERROR}
re-evaluating the initial task and correcting for the json error the correct json should be

// JSON:
"""

In [36]:
prompt_template.format(QUESTION="Create a cylinder with a radius of 5 and a height of 10")

'\n// Question : How do I add two numbers inside of Grasshopper?\n// Reasoning: To add two numbers we need to Add. There is an Addition component that performs this function. We need to create two numbers the user can edit, we can use the Number Slider for both numbers. And then we can Connections all of the components together\n// JSON:\n{\n\t"Advice": "Make sure to set the number sliders to the correct value",\n\t"Additions": [\n\t\t{\n\t\t\t"Name": "Number Slider",\n\t\t\t"Id": 1, // This number must be unique\n\t\t\t"value": "0..25..100" // This is optional\n\t\t},\n\t\t{\n\t\t\t"Name": "Number Slider",\n\t\t\t"Id": 2,\n\t\t\t"value": "-50..25..100" // This is optional\n\t\t},\n\t\t{\n\t\t\t"Name": "Addition",\n\t\t\t"Id": 3\n\t\t}\n\t],\n\t"Connections": [\n\t\t{\n\t\t\t"To": { // Addition\n\t\t\t\t"Id": 3,\n\t\t\t\t"ParameterName": "A"\n\t\t\t},\n\t\t\t"From": { // Number Slider,\n\t\t\t\t"Id": 1,\n\t\t\t\t"ParameterName": "number"\n\t\t\t}\n\t\t},\n\t\t{\n\t\t\t"To": { // Additi

## Baseline Call

In [37]:
QUESTION = "Create a cone with a base radius of 5 and a height of 10. Use number sliders not sliders"

In [38]:
# model: str = "gpt-3.5-turbo-1106"
model: str = "gpt-3.5-turbo"
temperature: float = 0

#prompt=prompt_template.format(QUESTION="Create a cylinder with a radius of 5 and a height of 10")
prompt=prompt_template.format(QUESTION=QUESTION)
#system_prompt=""

completion = call_openai(prompt=prompt, system_prompt=system_prompt, model=model, temperature=temperature)
result = completion.choices[0].message
print(result.content)

To create a cone with a base radius of 5 and a height of 10 using number sliders in Grasshopper, you can follow these steps:

1. Add a "Number Slider" component for the base radius.
2. Add a "Number Slider" component for the height.
3. Add a "Cone" component.
4. Connect the number slider for the base radius to the "Radius" input of the Cone component.
5. Connect the number slider for the height to the "Height" input of the Cone component.

JSON:
```json
{
	"Advice": "Adjust the number sliders to set the base radius and height accordingly",
	"Additions": [
		{
			"Name": "Number Slider",
			"Id": 1,
			"Value": "0..10..20"
		},
		{
			"Name": "Number Slider",
			"Id": 2,
			"Value": "0..10..20"
		},
		{
			"Name": "Cone",
			"Id": 3
		}
	],
	"Connections": [
		{
			"To": {
				"Id": 3,
				"ParameterName": "Radius"
			},
			"From": {
				"Id": 1,
				"ParameterName": "number"
			}
		},
		{
			"To": {
				"Id": 3,
				"ParameterName": "Height"
			},
			"From": {
				"Id": 2,
				"Paramet

# Instructor

In [39]:
os.environ["LANGCHAIN_PROJECT"] = f"GHPT_Instructor"

In [40]:
!pip install instructor

In [41]:
from pydantic import BaseModel, Field
from typing import List, Optional

class Component(BaseModel):
    Name: str = Field(..., description="The name of the component to be added. Only standard grasshopper components are allowed")
    Id: int = Field(..., description="A unique identifier for the component, starting from 1 and counting upwards")
    Value: Optional[str] = Field(None, alias='Value', description="The range of values for the component, if applicable. Only to be used for Panel, Number Slider, or Point components")

class InputConnectionDetail(BaseModel):
    Id: int = Field(..., description="The unique identifier of the component the connection is related to")
    ParameterName: str = Field(..., description="The specific input parameter of the component that the connection affects")

class OutputConnectionDetail(BaseModel):
    Id: int = Field(..., description="The unique identifier of the component the connection is related to")
    ParameterName: str = Field(..., description="The specific output parameter of the component that the connection affects")

class Connection(BaseModel):
    From: OutputConnectionDetail = Field(..., description="The source component and parameter from which the connection originates")
    To: InputConnectionDetail = Field(..., description="The target component and parameter that the connection is directing to")

class GrasshopperScriptModel(BaseModel):
    """
    A representation of a grasshopper script with all grasshopper components and the connections between them. Use Number Sliders for variable inputs to the script
    """
    ChainOfThought: str = Field(..., description="step by step rational explaining how the script acheives the aim, including the main components used")
    Advice: str = Field(..., description="A piece of advice or instruction related to using the grasshopper script")
    Additions: List[Component] = Field(..., description="A list of components to be added to the configuration")
    Connections: List[Connection] = Field(..., description="A list of connections defining relationships between components' parameters")

# Example usage:
example_json = {
    "ChainOfThought": "Use the sphere component with a slider as input to set the radius",
    "Advice": "Adjust the radius of the sphere using a number slider for desired size",
    "Additions": [
        {
            "Name": "Sphere",
            "Id": 1
        },
        {
            "Name": "Number Slider",
            "Id": 2,
            "Value": "5..50..100"
        }
    ],
    "Connections": [
        {
            "To": {
                "Id": 1,
                "ParameterName": "Radius"
            },
            "From": {
                "Id": 2,
                "ParameterName": "Number"
            }
        }
    ]
}

config = GrasshopperScriptModel(**example_json)
# print(example_json)
print(config.json())


{"ChainOfThought":"Use the sphere component with a slider as input to set the radius","Advice":"Adjust the radius of the sphere using a number slider for desired size","Additions":[{"Name":"Sphere","Id":1,"Value":null},{"Name":"Number Slider","Id":2,"Value":"5..50..100"}],"Connections":[{"From":{"Id":2,"ParameterName":"Number"},"To":{"Id":1,"ParameterName":"Radius"}}]}


In [42]:
!pip install anthropic
import instructor

client = instructor.from_openai(wrap_openai(OpenAI(api_key=OPENAI_API_KEY)))

In [43]:
# create the openai api call function
def call_openai(prompt: str, system_prompt: str = "", model: str = "gpt-3.5-turbo-1106", temperature: float = 0):
    completion = client.chat.completions.create(
        model=model,
        temperature=temperature,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        response_model=GrasshopperScriptModel
    )

    return completion

In [44]:
# DEFINE PROMPT HERE

#prompt = "Create a cylinder with a radius of 5 and a height of 10 and make sure to also create number sliders and connect them to the cylinder component"
prompt = QUESTION

In [45]:
model = "gpt-3.5-turbo-1106"
# model = "gpt-4-1106-preview	"
response = call_openai(prompt=prompt, system_prompt=system_prompt)

In [46]:
print(response.json())

{"ChainOfThought":"Create a Cone component and connect number sliders for base radius and height to its input parameters.","Advice":"Use number sliders to input the base radius and height values for the cone.","Additions":[{"Name":"Number Slider","Id":1,"Value":"5"},{"Name":"Number Slider","Id":2,"Value":"10"}],"Connections":[{"From":{"Id":1,"ParameterName":"Value"},"To":{"Id":3,"ParameterName":"Radius"}},{"From":{"Id":2,"ParameterName":"Value"},"To":{"Id":3,"ParameterName":"Height"}}]}
